In [1]:
import os
import time
import numpy as np
import pandas as pd
from IPython.display import clear_output 
from datetime import date

In [2]:
new_train_correlations = True ### will run for 10 hours for data from 1970 for all symbols, will delete old data
calc_predict_correlations =False ## will create new correlations for the 'periods' after deleting old data
update_train_correlations = False ## update train correlations for the missing dates if all tickers exist
remove_days = False
remove_after = '2022-07-01'
period = 30 # number of days to calculate predict correlations
data_file = '../../prices/adjclose.csv' # correlations on adjclose
train_save_path = '../processed/'
predict_save_path = '../data/predict/'

In [3]:
redo_date = False
today =str(date.today())
redo_date_list = [today]
redo_date_list = ['2022-04-04','2022-04-05','2022-04-06','2022-04-07']

In [4]:
with open('../../sp500tickers.txt','r') as f:
    symbols=[line.rstrip('\n') for line in f]

In [5]:
df=pd.read_csv(data_file, index_col=0, parse_dates=True)[:]
#df = df[:'2021-08-19']
df.tail()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,ZTS,^GSPC,^IXIC,^NDX,^RUT,^SOX,^TNX,^VIX,^VVIX,^VXN
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-08,133.380005,14.58,185.559998,159.902237,140.080002,111.859612,77.120003,340.350006,580.549988,188.979996,...,196.149994,4763.540039,14843.769531,16649.869141,1989.010010,4062.479980,4.002,13.08,81.260002,16.860001
2024-01-09,130.679993,14.38,185.139999,160.773911,139.529999,112.188026,76.150002,342.750000,586.200012,189.759995,...,195.940002,4756.500000,14857.709961,16678.699219,1968.040039,4064.649902,4.019,12.76,78.589996,16.549999
2024-01-10,131.089996,14.35,186.190002,163.329498,139.759995,113.849998,76.959999,345.329987,591.030029,188.369995,...,198.770004,4783.450195,14969.650391,16793.050781,1970.260010,4048.840088,4.030,12.69,77.089996,16.600000
2024-01-11,129.679993,14.59,185.589996,162.229996,139.449997,113.500000,77.620003,347.970001,597.489990,188.960007,...,196.710007,4780.240234,14970.190430,16820.900391,1955.459961,4067.899902,3.977,12.44,75.540001,16.400000
2024-01-12,130.539993,13.21,185.919998,162.399994,137.139999,113.919998,77.690002,356.329987,596.539978,188.360001,...,198.940002,4783.830078,14972.759766,16832.919922,1950.959961,4052.429932,3.950,12.70,86.070000,16.540001


In [6]:
def train_correlations(df):
    for i in df.index[::1]:
        date=pd.to_datetime(i).date()
        time.sleep(.1); clear_output()
        print(date)
        corr=df[pd.to_datetime(df.index[0]).date():i].corr()
        for symbol in symbols:
            df1 = corr.drop(symbol, axis=1)
            df1=df1.loc[[symbol],:]
            df1.rename(index={symbol:date},inplace=True)

            if not os.path.exists(train_save_path+symbol+'.csv'):
                df1=df1.add_prefix('corr'+symbol+'_')
                df1.to_csv(train_save_path+symbol+'.csv')
            else:
                df1.to_csv(train_save_path+symbol+'.csv', mode='a', header = False)

            del df1
        del corr    

In [7]:
def predict_correlations(df, period):
    for i in df.index[:-period:1]:
        date=pd.to_datetime(i).date()
        time.sleep(.1); clear_output()
        print(date)
        corr=df[pd.to_datetime(df.index[0]).date():i].corr()
        for symbol in symbols:
            df1 = corr.drop(symbol, axis=1)
            df1=df1.loc[[symbol],:]
            df1.rename(index={symbol:date},inplace=True)

            if not os.path.exists(predict_save_path+symbol+'.csv'):
                df1=df1.add_prefix('corr'+symbol+'_')
                df1.to_csv(predict_save_path+symbol+'.csv')
            else:
                df1.to_csv(predict_save_path+symbol+'.csv', mode='a', header = False)

            del df1
        del corr    

In [8]:
%%time
if new_train_correlations:# uncommented to prevent deleting data by mistake or generate duplicate data
    for symbol in symbols:
        if os.path.exists(train_save_path+symbol+'.csv'):
            os.remove(train_save_path+symbol+'.csv') 
    train_correlations(df)
    df = pd.read_csv(train_save_path+'^GSPC.csv', index_col=0)
    df=df.sort_index()
    print(df.iloc[-5:,0:4]) # display 5 rows and 4 columns
    del df

2024-01-12
            corr^GSPC_A  corr^GSPC_AAL  corr^GSPC_AAPL  corr^GSPC_ABBV
2024-01-08     0.970794       0.093347        0.895518        0.930056
2024-01-09     0.970805       0.092972        0.895603        0.930155
2024-01-10     0.970815       0.092592        0.895690        0.930255
2024-01-11     0.970822       0.092221        0.895777        0.930355
2024-01-12     0.970830       0.091803        0.895864        0.930455
Wall time: 15h 42min 17s


In [9]:
if calc_predict_correlations:
    for symbol in symbols:
        if os.path.exists(predict_save_path+symbol+'.csv'): 
            os.remove(predict_save_path+symbol+'.csv')  
    predict_correlations(df, period)
    df = pd.read_csv(predict_save_path+'^GSPC.csv', index_col=0)
    df=df.sort_index()
    print(df.iloc[-5:,0:4]) # display 5 rows and 4 columns
    del df

In [10]:
def find_download_dates():
    df_date = []
    for i in df.index[::-1]:
        df_date.append(pd.to_datetime(i).date())
        
    existing_date = []
    symbol = '^GSPC'
    existing = pd.read_csv(train_save_path+symbol+'.csv', index_col=0)
    for j in existing.index[::]:
        existing_date.append(pd.to_datetime(j).date())
    
    download_date = []
    for k in df_date:
        if k in existing_date:
            break
        else:
            download_date.append(k)
    del df_date, existing_date, existing
    return download_date 

In [11]:
# %%time
# if update_train_correlations:
#     download_date = ['2021-08-03']
#     for i in download_date:
#         date=pd.to_datetime(i).date()
#         time.sleep(.1); clear_output()
#         print(date)
#         corr=df[pd.to_datetime(df.index[0]).date():i].corr()
#         for symbol in symbols[:]:
#             df1 = corr.drop(symbol, axis=1)
#             df1=df1.loc[[symbol],:]
#             df1.rename(index={symbol:date},inplace=True)
#             df1.to_csv(train_save_path+symbol+'.csv', mode='a', header = False)
#     # sample view of resulting data
#     df=pd.read_csv(train_save_path+'^GSPC'+'.csv', parse_dates=True, index_col=0)
#     df=df.sort_index()
#     print(df.iloc[-5:,0:4]) # display 5 rows and 4 columns
#     del df

In [12]:
%%time
if update_train_correlations:
    if redo_date:
        download_date = redo_date_list
    else:
        download_date = find_download_dates()
    for i in download_date:
        date=pd.to_datetime(i).date()
        time.sleep(.1); clear_output()
        print(date)
        corr=df[pd.to_datetime(df.index[0]).date():i].corr()
        for symbol in symbols[:]:
            df1 = corr.drop(symbol, axis=1)
            df1=df1.loc[[symbol],:]
            df1.rename(index={symbol:date},inplace=True)
            df1.to_csv(train_save_path+symbol+'.csv', mode='a', header = False)
    del df

Wall time: 0 ns


In [13]:
# remove_days = True
# symbols =['^GSPC']
# remove_after = '2022-01-07'
if remove_days:
    for count,symbol in enumerate(symbols[460:]):
        df = pd.read_csv(train_save_path+symbol+'.csv', index_col=0).sort_index()
        df=df[~df.index.duplicated(keep='last')]
        df = df[:remove_after]
        df.to_csv(train_save_path+symbol+'.csv')
        del df
        print(count, symbol)   

In [14]:
df=pd.read_csv(train_save_path+'^GSPC'+'.csv', parse_dates=True, index_col=0).sort_index()
df.to_csv(train_save_path+'^GSPC'+'.csv')
print(df.iloc[-40:,0:4])

            corr^GSPC_A  corr^GSPC_AAL  corr^GSPC_AAPL  corr^GSPC_ABBV
2023-11-15     0.970271       0.107383        0.893221        0.926804
2023-11-16     0.970256       0.106958        0.893273        0.926883
2023-11-17     0.970237       0.106535        0.893326        0.926963
2023-11-20     0.970218       0.106110        0.893379        0.927042
2023-11-21     0.970226       0.105679        0.893434        0.927122
2023-11-22     0.970234       0.105250        0.893489        0.927202
2023-11-24     0.970247       0.104822        0.893547        0.927282
2023-11-27     0.970257       0.104392        0.893604        0.927362
2023-11-28     0.970266       0.103961        0.893661        0.927440
2023-11-29     0.970281       0.103534        0.893719        0.927519
2023-11-30     0.970296       0.103110        0.893778        0.927604
2023-12-01     0.970311       0.102702        0.893838        0.927693
2023-12-04     0.970328       0.102309        0.893899        0.927780
2023-1